In [2]:
import pandas as pd

In [62]:
df = pd.read_csv('/workspaces/vikrant_dubai/final/AD/AD 1.csv')
df.head(10)

,Station Name,Date,Time,SO2,Data Cap. (SO2),SO2 Avg,NO2,Data Cap. (NO2),NO2 Avg,O3,...,Ethylbenzene,MP-Xylene,Benzene,CH4,NMHC,THC,Noise,Solar Radiation,Atm P.,DATA COUNT
0,Baniyas School,01/01/2023,00:00:00,4.13,NaN,6.45,66.33,NaN,44.88,21.15,...,NaN,NaN,NaN,NaN,NaN,NaN,52.97,NaN,NaN,NaN
1,Baniyas School,01/01/2023,01:00:00,4.81,NaN,6.45,74.41,NaN,44.88,11.71,...,NaN,NaN,NaN,NaN,NaN,NaN,59.76,NaN,NaN,NaN
2,Baniyas School,01/01/2023,02:00:00,5.11,NaN,6.45,71.63,NaN,44.88,9.04,...,NaN,NaN,NaN,NaN,NaN,NaN,52.66,NaN,NaN,NaN
3,Baniyas School,01/01/2023,03:00:00,6.09,NaN,6.45,76.44,NaN,44.88,5.66,...,NaN,NaN,NaN,NaN,NaN,NaN,53.57,NaN,NaN,NaN
4,Baniyas School,01/01/2023,04:00:00,5.46,NaN,6.45,69.75,NaN,44.88,9.62,...,NaN,NaN,NaN,NaN,NaN,NaN,52.54,NaN,NaN,NaN
5,Baniyas School,01/01/2023,05:00:00,5.81,NaN,6.45,68.02,NaN,44.88,5.57,...,NaN,NaN,NaN,NaN,NaN,NaN,51.84,NaN,NaN,NaN
6,Baniyas School,01/01/2023,06:00:00,7.95,NaN,6.45,59.16,NaN,44.88,5.10,...,NaN,NaN,NaN,NaN,NaN,NaN,57.24,NaN,NaN,NaN
7,Baniyas School,01/01/2023,07:00:00,9.18,NaN,6.45,53.94,NaN,44.88,5.71,...,NaN,NaN,NaN,NaN,NaN,NaN,52.77,NaN,NaN,NaN
8,Baniyas School,01/01/2023,08:00:00,9.08,NaN,6.45,47.93,NaN,44.88,6.77,...,NaN,NaN,NaN,NaN,NaN,NaN,55.10,NaN,NaN,NaN
9,Baniyas School,01/01/2023,09:00:00,8.43,NaN,6.45,59.02,NaN,44.88,6.89,...,NaN,NaN,NaN,NaN,NaN,NaN,54.54,NaN,NaN,NaN


In [63]:
df.columns

Index(['Station Name', 'Date', 'Time', 'SO2', 'Data Cap. (SO2)', 'SO2 Avg',
       'NO2', 'Data Cap. (NO2)', 'NO2 Avg', 'O3', 'Data Cap. (O3)',
       'O3 8H Avg.', 'Daily Capture O3', 'O3 Daily Average',
       'O3 Daily Average (NS)', 'CO', 'Data Cap. (CO)', 'CO 8H Avg.',
       'Daily Capture CO', 'CO Daily Average', 'PM10', 'Data Cap. (PM10)',
       'PM10 Avg', 'PM2.5', 'Data Cap. (PM2.5)', 'PM2.5 Avg',
       'Lower Ambient Temperature', 'Upper Ambient Temperature',
       'Barometric Pressure', 'Relative Humidity', 'Wind Direction',
       'Wind Speed', 'H2S', 'Toluene', 'O-Xylene', 'Ethylbenzene', 'MP-Xylene',
       'Benzene', 'CH4', 'NMHC', 'THC', 'Noise', 'Solar Radiation', 'Atm P.',
       'DATA COUNT'],
      dtype='object')

In [66]:
unique_station_names = df['Station Name'].unique()
print(unique_station_names.shape)

(20,)


In [68]:
def create_finder_table(df):
    df['vlookup'] = df['Station Name'].str[:3] + df['Date'].str.replace('/', '')
    df_grouped = df.groupby(['Station Name', 'Date']).first().reset_index()
    df_grouped['vlookup'] = df_grouped['Station Name'].str[:5] + df_grouped['Date'].str.replace('/', '')
    Finder_table = df_grouped[['Station Name', 'Date', 'vlookup']]
    return Finder_table

# Example usage:
Finder_table = create_finder_table(df)
Finder_table

,Station Name,Date,vlookup
0,Al Ain Islamic Institute,01/01/2023,Al Ai01012023
1,Al Ain Islamic Institute,02/01/2023,Al Ai02012023
2,Al Ain Islamic Institute,03/01/2023,Al Ai03012023
3,Al Ain Islamic Institute,04/01/2023,Al Ai04012023
4,Al Ain Islamic Institute,05/01/2023,Al Ai05012023
...,...,...,...
615,Zakher,27/01/2023,Zakhe27012023
616,Zakher,28/01/2023,Zakhe28012023
617,Zakher,29/01/2023,Zakhe29012023
618,Zakher,30/01/2023,Zakhe30012023


In [67]:
pollutants = ['SO2', 'NO2','CO','PM10', 'PM2.5']
avg_values = df.groupby(['Station Name', 'Date'])[pollutants].mean().reset_index()
datacap_values = df.groupby(['Station Name', 'Date'])[pollutants].apply(lambda x: (x.isna().sum()/24)*100).reset_index()

final_df = pd.merge(avg_values, datacap_values, on=['Station Name', 'Date'], suffixes=(' Avg 24H', ' Data Cap.'))
final_df.rename(columns={'Station Name': 'Name'}, inplace=True)
final_df['vlookup'] = final_df['Name'].str[:3] + final_df['Date'].str.replace('/', '')
final_df.drop(columns=['Name', 'Date']).head(60)


,SO2 Avg 24H,NO2 Avg 24H,CO Avg 24H,PM10 Avg 24H,PM2.5 Avg 24H,SO2 Data Cap.,NO2 Data Cap.,CO Data Cap.,PM10 Data Cap.,PM2.5 Data Cap.,vlookup
0,5.345417,25.265000,NaN,64.845833,14.579167,0.000000,0.000000,100.000000,0.000000,0.000000,Al 01012023
1,5.411250,29.375417,NaN,62.404167,19.250000,0.000000,0.000000,100.000000,0.000000,0.000000,Al 02012023
2,6.594583,40.979167,NaN,73.141667,10.075000,0.000000,0.000000,100.000000,0.000000,0.000000,Al 03012023
3,5.446667,20.700833,NaN,81.520833,5.916667,0.000000,0.000000,100.000000,0.000000,0.000000,Al 04012023
4,5.048333,33.338750,NaN,85.054167,9.616667,0.000000,0.000000,100.000000,0.000000,0.000000,Al 05012023
5,5.646250,29.213750,NaN,71.716667,11.133333,0.000000,0.000000,100.000000,0.000000,0.000000,Al 06012023
6,6.037083,22.400000,NaN,77.891667,10.991667,0.000000,0.000000,100.000000,0.000000,0.000000,Al 07012023
7,6.835417,24.896250,NaN,37.000000,6.704167,0.000000,0.000000,100.000000,0.000000,0.000000,Al 08012023
8,5.695417,27.755833,NaN,40.704167,11.612500,0.000000,0.000000,100.000000,0.000000,0.000000,Al 09012023
9,5.787500,28.176250,NaN,398.179167,55.912500,0.000000,0.000000,100.000000,0.000000,0.000000,Al 10012023
